# EmpkinS Mikro Data Loader

In [1]:
from pathlib import Path

import re

import pandas as pd
import numpy as np
import pingouin as pg

from scipy.ndimage import shift
import scipy.signal as signal
from sklearn.metrics import mean_absolute_error

import librosa as lr

import biopsykit as bp
from biopsykit.signals.ecg import EcgProcessor

from empkins_io.sensors.radar import load_radar_data, sync_with_ecg, split_data, correct_outlier

from empkins_micro.time_logs import process_time_log
from empkins_micro.plotting import hr_plot_ecg_radar
from empkins_micro.heart_rate_validation import resample_data
from empkins_micro.signal_alignment import chisqr_align, ccovf, phase_align

import matplotlib.pyplot as plt
import seaborn as sns

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
palette = bp.colors.fau_palette
sns.set_theme(context="notebook", style="ticks", palette=palette)

figsize = (10, 5)

plt.close("all")
plt.rcParams['figure.figsize'] = figsize
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['mathtext.default'] = "regular"

pg.options['round'] = 3

palette

[(0.0, 0.2196078431372549, 0.396078431372549),
 (0.596078431372549, 0.6431372549019608, 0.6823529411764706),
 (0.788235294117647, 0.5764705882352941, 0.07450980392156863),
 (0.0, 0.6941176470588235, 0.9215686274509803),
 (0.0, 0.6078431372549019, 0.4666666666666667),
 (0.5529411764705883, 0.0784313725490196, 0.1607843137254902)]

In [3]:
subject_id = "VP_02"
condition = "stress"
study_part = "pre"

study_part_dict = {
    "pre": 0,
    "mist": 1,
    "post": 2
}

outlier_correction = ["statistical_rr", "statistical_rr_diff", "artifact", "physiological"]

In [4]:
base_path = Path("../../../../../HealthPsychology_D03/Data/2021_06_Micro_Prestudy")
data_path = base_path.joinpath("data_per_subject/{}/{}".format(subject_id, condition))

ecg_path = data_path.joinpath("ecg/raw")
radar_path = data_path.joinpath("radar/raw")
video_path = data_path.joinpath("video")
timelog_path = data_path.joinpath("time_log/processed")

In [5]:
ecg_files = list(sorted(ecg_path.glob("*.bin")))
radar_files = list(sorted(radar_path.glob("*.mat")))
video_files = list(sorted(video_path.glob("*.mp4")))

ecg_file = ecg_files[study_part_dict[study_part]]
radar_file = radar_files[study_part_dict[study_part]]
video_file = video_files[3]

timelog_file = timelog_path.joinpath("time_log_{}.csv".format(subject_id))
print("{}\t {}\t {}".format(ecg_file.name, radar_file.name, video_file.name))

NilsPodX-E18A_20210609_114647.bin	 2021-06-09_11-47-01_VP_02_EB_1__rawdata.mat	 VP_02_pre_H.264.mp4


## Load NilsPod and Radar Data

In [6]:
ecg, fs_ecg = bp.io.nilspod.load_dataset_nilspod(ecg_file)
radar, fs_radar = load_radar_data(radar_file, datastreams=["hr"])
radar = radar["hr"]

In [7]:
radar, ecg = sync_with_ecg(radar, ecg)

ecg["ecg"]= -1 * ecg["ecg"]

In [8]:
timelog = bp.io.load_time_log(timelog_file, continuous_time=False)
timelog = timelog.filter(like=study_part)

In [9]:
ep = EcgProcessor(data=ecg, sampling_rate=fs_ecg, time_intervals=timelog)
ep.ecg_process(outlier_correction=outlier_correction)
ecg_dict = ep.rpeaks

  0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
radar_dict = split_data(radar, timelog.loc[subject_id])
radar_dict = {key: correct_outlier(value, outlier_correction=outlier_correction, sampling_rate=fs_radar) for key, value in radar_dict.items()}

In [11]:
key = "pre_baseline"

hr_ecg = ecg_dict[key]
hr_radar = radar_dict[key]
imu_data = ecg.loc[hr_ecg.index[0]:hr_ecg.index[-1]]

### Overview Plot

In [12]:
fig, axs = bp.signals.ecg.plotting.ecg_plot(ep, key=key, figsize=figsize)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Chest IMU Data

In [14]:
#from gaitmap.preprocessing import align_dataset_to_gravity

In [15]:
#acc_data_align = align_dataset_to_gravity(imu_data, sampling_rate_hz=fs_ecg).filter(like="acc")

# remove gravity
#acc_data_align["acc_z"] -= 9.81
#acc_energy_align = pd.DataFrame((acc_data_align**2).sum(axis=1), columns=["acc_energy"])

In [16]:
#fig, axs = plt.subplots(ncols=2)
#acc_data_align.rolling(100).mean().plot(ax=axs[0], title="3-axis Acceleration")
#acc_energy_align.rolling(100).mean().plot(ax=axs[1], title="Acceleration Energy")

## Audio Data

In [17]:
audio_data, fs_audio = lr.load(video_file)
ds = 100
fs_downsample = fs_audio / ds
audio_downsample = audio_data[::ds]

/Users/Richer/Documents/PhD/Projects/EmpkinS/Code/empkins_base_kernel/.venv/lib/python3.8/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [18]:
audio_downsample = pd.DataFrame(audio_downsample, index=hr_ecg.index[0] + pd.TimedeltaIndex(np.arange(0, len(audio_downsample)) / fs_downsample, unit='s'))
audio_downsample = np.abs(audio_downsample)
audio_downsample = audio_downsample.loc[hr_ecg.index[0]:hr_ecg.index[-1]]

In [19]:
#acc_roll = acc_energy_align.rolling(100).mean()

In [20]:
fig, ax = plt.subplots(figsize=figsize)
fig, ax = hr_plot_ecg_radar(hr_ecg=hr_ecg, hr_radar=hr_radar, plot_radar_quality=True, plot_outlier=False, plot_mean=False, ax=ax)

ylims = ax.get_ylim()

ymax = (ylims[1] - ylims[0]) * 0.5 + ylims[0]
ylims = [ylims[0], ymax]

#acc_roll_rescaled = (ylims[1] - ylims[0]) * (acc_roll - acc_roll.min()) / (acc_roll.max() - acc_roll.min()) + ylims[0]

#audio_rescaled = (ylims[1] - ylims[0]) * (audio_downsample - audio_downsample.min()) / (audio_downsample.max() - audio_downsample.min()) + ylims[0]

#h = ax.plot(acc_roll_rescaled, color=bp.colors.fau_color("tech"), alpha=0.3)
#l = ax.legend(h, ["Acc. Energy"], loc="lower left", fontsize="small")
#ax.add_artist(l)
#h = ax.plot(audio_rescaled, color=bp.colors.fau_color("phil"), alpha=0.3)
#l = ax.legend(h, ["Audio Signal"], loc="lower right", fontsize="small")
#ax.add_artist(l)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
resample_rate = 10
hr_ecg_res = resample_data(hr_ecg["Heart_Rate"], resample_rate=resample_rate)
hr_radar_res = resample_data(hr_radar["Heart_Rate"], resample_rate=resample_rate)

index_intersect = hr_ecg_res.index.intersection(hr_radar_res.index)
hr_ecg_res = hr_ecg_res.loc[index_intersect]
hr_radar_res = hr_radar_res.loc[index_intersect]

shift_idx = phase_align(np.squeeze(hr_ecg_res), np.squeeze(hr_radar_res), roi=[0, len(hr_ecg_res)])
hr_radar_res = pd.DataFrame(shift(hr_radar_res, shift=shift_idx, mode="nearest"), columns=hr_radar_res.columns, index=hr_radar_res.index)

print("Radar Shift: {}".format(shift_idx))

Radar Shift: -5.73


In [35]:
fig, ax = plt.subplots(figsize=figsize)
fig, ax = hr_plot_ecg_radar(hr_ecg=hr_ecg, hr_radar=hr_radar, plot_outlier=False, plot_radar_quality=False, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
fig, ax = plt.subplots(figsize=figsize)
fig, ax = hr_plot_ecg_radar(hr_ecg=hr_ecg_res, hr_radar=hr_radar_res, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
display(pg.corr(np.squeeze(hr_ecg_res), np.squeeze(hr_radar_res)))
print("Mean Absolute Error: {:.03f} bpm".format(mean_absolute_error(hr_ecg_res, hr_radar_res)))

,n,r,CI95%,p-val,BF10,power
pearson,1191,0.479,"[0.43, 0.52]",0.0,7.278e+65,1.0


Mean Absolute Error: 2.543 bpm


## Sensor Data Agreement

In [24]:
x_data = hr_ecg_res["Heart_Rate"][::10]
x_data.name = "$HR_{ECG}$ [bpm]"
y_data = hr_radar_res["Heart_Rate"][::10]
y_data.name = "$HR_{Radar}$ [bpm]"

In [25]:
llim = np.min([np.min(x_data), np.min(y_data)])
rlim = np.max([np.max(x_data), np.max(y_data)])

g = sns.jointplot(
    x=x_data, y=y_data, 
    #xlim=[llim, rlim], ylim=[llim, rlim], 
    kind="reg"
)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
fig, ax = plt.subplots(figsize=figsize)
ax = pg.plot_blandaltman(x=x_data, y=y_data, scatter_kws={"color": bp.colors.fau_color("fau"), "alpha": 0.8}, ax=ax)
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …